In [ ]:
import cv2
import pickle
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Activation, Input, Embedding, LSTM, Dense, merge, Convolution2D, MaxPooling2D, Reshape, Flatten, Dropout
from keras.optimizers import RMSprop, Adam
from keras import initializations

In [ ]:
# Util functions
def my_init(shape, name=None):
    return initializations.normal(shape, scale=0.01, name=name)

def augment_brightness_camera_images(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25+np.random.uniform()
    #print(random_bright)
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

In [ ]:
# TODO: Fill this in based on where you saved the training and testing data

training_file = '/home/autti/Downloads/traffic-signs-data/train_p2.p'
testing_file = '/home/autti/Downloads/traffic-signs-data/test_p2.p'


with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)


X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [ ]:
print("shape train", X_train.shape)
print(X_train[0, :, :, 0])
print("labels", y_train.shape)
plt.figure()
plt.imshow(X_train[0, :, :, :])

In [ ]:
# normalize data
X_train = X_train.astype(np.float32)/255
X_train -= 0.5
print(X_train[0,:,:,0])
print("max",np.max(y_train),"min",np.min(y_train))
y_label = np.zeros(shape=(y_train.shape[0], np.max(y_train) + 1))

for i in range(y_train.shape[0]):
    y_label[i, y_train[i]]=1

#X_train = X_train[:10]
#y_label = y_label[:10]
#print(y_label[-2:,:])

In [ ]:
prefix = 'traffic'
model_path ='.model_{0}.h5'.format(prefix) 
sgd_select = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [ ]:
# network architecture

# input layer
main_input = Input(shape=(32, 32, 3), name='main_input')

color_map = Convolution2D(3, 1, 1, init=my_init, border_mode='same')(main_input)

conv1 = Convolution2D(32, 3, 3, init=my_init, border_mode='same')(color_map)
conv2 = Convolution2D(32, 3, 3, init=my_init, border_mode='same')(conv1)
pool1 = MaxPooling2D(pool_size=(3, 3))(conv2)
drop1 = Dropout(0.5)(pool1)

conv3 = Convolution2D(64, 3, 3, init=my_init, border_mode='same')(drop1)
conv4 = Convolution2D(64, 3, 3, init=my_init, border_mode='same')(conv3)
pool2 = MaxPooling2D(pool_size=(3, 3))(conv4)
drop2 = Dropout(0.5)(pool2)

conv5 = Convolution2D(128, 3, 3, init=my_init, border_mode='same')(drop2)
conv6 = Convolution2D(128, 3, 3, init=my_init, border_mode='same')(conv5)
pool3 = MaxPooling2D(pool_size=(3, 3))(conv6)
drop3 = Dropout(0.5)(pool3)

# flatten layer
flat1 = Flatten()(drop1)
flat2 = Flatten()(drop2)
flat3 = Flatten()(drop3)

# concatenate all vectors
merged1 = merge([flat1, flat2, flat3], name='merge1', mode='concat',concat_axis = 1)

fc1 = Dense(1024)(merged1)
drop4 = Dropout(0.5)(fc1)

fc2 = Dense(1024)(drop4)
drop5 = Dropout(0.5)(fc2)

softmax_layer = Dense(43)(drop5)
main_output = Activation('softmax')(softmax_layer)
print("final", main_output)

In [ ]:
# select inputs, outputs and customized sgd
sign_model = Model(input = [main_input], output = [main_output])
sign_model.compile(optimizer = sgd_select, loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
sign_model.fit([X_train], [y_label], nb_epoch=50, batch_size=256)
sign_model.save(model_path)